## MTCNN & CNN & system video

In [2]:
import cv2
import requests
from io import BytesIO
# extract and plot each detected face in a photograph
from matplotlib import pyplot
from matplotlib.patches import Rectangle,Circle
from mtcnn import MTCNN
# from mtcnn.mtcnn import MTCNN
# from google.colab.patches import cv2_imshow
import zipfile
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder,Normalizer
from sklearn.svm import SVC
from random import choice
from PIL import Image as Img
from numpy import savez_compressed,asarray,load,expand_dims
from keras.models import load_model
# from keras_facenet import FaceNet
import tensorflow as tf
from tensorflow.keras import layers, models
import time

In [4]:
# class Data_Using:

def unzip(zip_dir,out_dir):
    path_folder = out_dir

    zip_file_path = f'{zip_dir}'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(path_folder)


def load_dataset(directory):
  	# enumerate folders, on per class
  images = []
  path_d = []
  allpath = []
  y = []
  for subdir in os.listdir(directory):

    # print(y)
       	# enumerate files
    for filename in os.listdir(directory+subdir+'/'):
      path = directory + subdir + '/' + filename
          #labels
      y.append(subdir)
      path_d.append(path)
      # print(path_d)
  # allpath.extend(path_d)

  print(path_d)
	  	# store
    # images.append(path_d)
  return path_d , y


def load_data_image(path_image):
    img = cv2.imread(path_image)
    # img = cv2.cvtColor(img,cv2.COLOR)
    return img


def load_data_video(path_video):

    frame_list = []
    # Load the video content using OpenCV
    cap = cv2.VideoCapture(path_video)
    # Loop through the frames and display them
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Display the frame
        # cv2_imshow( frame)

        frame_list.append(frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()
    frame_list = np.array(frame_list)
    return frame_list



def load_data_system_camera():
    # define a video capture object
    vid = cv2.VideoCapture(0)
    while(True):
        # Capture the video frame
        ret, frame = vid.read()
        # Display the resulting frame
        cv2.imshow('frame', frame)
        # cv2.imwrite('video.mp4',frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()



In [5]:
from functools import wraps
import sys
import io
def capture_output(func):
    """Wrapper to capture print output."""

    @wraps(func)
    def wrapper(*args, **kwargs):
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        try:
            return func(*args, **kwargs)
        finally:
            sys.stdout = old_stdout

    return wrapper

In [6]:
class Face_Detection_Recognition_Video:

    def __init__(self):
      # super.__init__(self)
      self.face_number = 0
      self.folder_number = 0
      self.detector = MTCNN()
      self.w_detect = capture_output( self.detector.detect_faces)

    # def preprocess(self,image):
    #   #normalize
    #     pixels = image.astype('float32')
    #     pixels /= 255.0
    #     return pixels

      # draw each face separately
    def mtcnn_model(self,data, result_list):
        # get the context for drawing boxes
        # ax = pyplot.gca()
        # for result in result_list:
        #   # get coordinates
        #   x, y, width, height = result['box']
        #   # create the shape
        #   rect = Rectangle((x, y), width, height, fill=False, color='red')
        #   # draw the box
        #   ax.add_patch(rect)
        #   # draw the dots
        #   for key, value in result['keypoints'].items():
        #   # create and draw dot
        #     dot = Circle(value, radius=2, color='red')
        #     ax.add_patch(dot)
        # pyplot.show()
        crop_face = []
        for i in range(len(result_list)):
          # get coordinates
          x1, y1, width, height = result_list[i]['box']
          x2, y2 = x1 + width, y1 + height
          # define subplot
          # pyplot.subplot(1, len(result_list), i+1)
          # pyplot.axis('off')
          # plot face
          Face = data[y1:y2, x1:x2]
          # resize pixels to the model size
          Face = Img.fromarray(Face)
          Face = Face.convert('RGB')
          Face = Face.resize((160,160))
          Face = asarray(Face)#,dtype=np.float32)
          crop_face.append(Face)

        return crop_face


    def get_face(self,path):
        pixels = path
        pixels = asarray(pixels)
        # detector = MTCNN()
        # detect faces in the image
        # w_detect = capture_output( self.detector.detect_faces)
        start=time.time()
        faces_pos = self.w_detect(pixels)
        stop=time.time()
        # start=time.time()
        Faces = self.mtcnn_model(pixels, faces_pos)
        stop_1=time.time()
        print('detect_time:',stop-start)
        print('all_time:',stop_1-start)
        # print(stop)

        return Faces , faces_pos


    def model_CNN(self):
      model_cnn = models.Sequential([
      layers.Conv2D(16, (3, 3), activation='relu', input_shape=(160, 160, 3)),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(32, (3, 3), activation='relu'),# input_shape=(160, 160, 3)),
      layers.MaxPooling2D((2, 2)),
      # layers.Dropout(0.2),
      layers.Conv2D(64, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      # layers.Dropout(0.2),
      layers.Conv2D(128, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Flatten(),
      layers.Dense(512, activation='relu'),
      layers.Dropout(0.2),  # Dropout for regularization
      layers.Dense(128, activation='relu'),
      # layers.Dropout(0.2),
      # layers.Dense(64, activation='relu'),
      layers.Dense(9, activation='softmax')  # num_classes is the number of identities
      ])

      return model_cnn

    def train_cnn_model(self,trainx,trainy,testx, testy,E_number):
      self.model = self.model_CNN()
      self.model.summary()
      self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
      self.model.fit(trainx, trainy, epochs= E_number)
      test_loss, test_acc = self.model.evaluate(testx, testy)
      return self.model , test_loss, test_acc


    def recognition_model(self,image,model):
      random_face = np.array(image)
      random_face = random_face.reshape(1,160,160,3)
      w_predict = capture_output(model.predict)
      pred = w_predict(random_face)
      # random_face_name = out_encoder.inverse_transform([random_face_class])
      out_encoder = LabelEncoder()
      out_encoder.fit(trainy)
      class_index = tf.argmax(pred, axis=1)
      # confidence = tf.reduce_max(pred[0][random_face_class])
      pred_name = out_encoder.inverse_transform(class_index)
      # print('Predicted: %s' % pred_name)
      # print('Expected: %s' % random_face_name[0])
      # print(f"Predicted class: {class_index}, Confidence: {confidence * 100:.2f}%")
      # pyplot.figure()
      # pyplot.imshow(random_face.reshape(160,160,3))
      # pyplot.show()
      return pred_name

    def save_faces(self,data_face):
      folder_name = 'Faces'+f'{self.folder_number}'
      if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
      cv2.imwrite(folder_name + '/' + f'{self.face_number}.jpg',data_face)
      self.face_number += 1

    def face_counting(self,):
      pass

In [7]:
path = 'Facedata.zip'
output = '/Users/zahrad/Downloads/testvideo1/Facedata/photos'
data = unzip(path,output)

In [8]:
!rm -R /Users/zahrad/Downloads/testvideo1/Facedata/photos/__MACOSX

In [15]:
!find . -name "*.DS_Store" -type f -delete

In [16]:
# Load dataset

def faces_dataset(directory):
  X = list()
  newlabels = list()
  files , labels = load_dataset(directory)
  print(len(files),len(labels))
  for fil,label in zip(files,labels):
    facedetect = Face_Detection_Recognition_Video()
    img = load_data_image(fil)
    if not type(img) == type(None):
     faces , pos = facedetect.get_face(img)
    if not len(faces)== 0:
      X.append(faces[0])
      newlabels.append(label)

  return asarray(X), asarray(newlabels)



# paths train and validation
train_path = '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/'
val_path = '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/val/'
# load train dataset
trainX, trainy = faces_dataset(train_path)
# load test dataset
testX, testy = faces_dataset(val_path)
# save arrays to one file in compressed format
# savez_compressed('5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

['/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/me.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20230511_142814.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20230511_143922.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_210541.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_210420.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_210556.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_210609.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20230511_140800.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_212247.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20230511_150134.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/

In [18]:
# Prepare data for Cnn

Cnn_trainx = []
for p in trainX:
  Cnn_trainx.append(p[0])
Cnn_trainx = np.array(Cnn_trainx)
print(Cnn_trainx.shape)

Cnn_testx = []
for p in testX:
  Cnn_testx.append(p[0])
Cnn_testx = np.array(Cnn_testx)
print(Cnn_testx.shape)

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
train_y = out_encoder.transform(trainy)
print(train_y.shape)
test_y = out_encoder.transform(testy)
print(test_y.shape)

(230, 160, 3)
(44, 160, 3)
(230,)
(44,)


In [10]:
# Train model
# face_detect = Face_Detection_Recognition_Video()
# train_model = face_detect.train_cnn_model(Cnn_trainx,train_y,Cnn_testx,test_y,100)

In [11]:
# train_model[0].save_weights('/content/drive/MyDrive/Facenet/Model/weightmodel')
# train_model[0].save('/content/drive/MyDrive/Facenet/Model/Mymodel.h5')

In [19]:
# Loads the weights
face_detect = Face_Detection_Recognition_Video()
new_model = tf.keras.models.load_model('/Users/zahrad/Downloads/Model/Mymodel.h5')
# model = face_detect.model_CNN()
# new_model.load_weights('/Users/zahrad/Downloads/Model/checkpoint')
# Load the previously saved weights
# new_model.load_weights('/Users/zahrad/Downloads/Model/weightmodel.index')

In [20]:
# start streaming video from webcam
# video_stream()
# label for video
# label_html = 'Capturing...'
# initialze bounding box to empty
# bbox = ''
# count = 0
video_capture = cv2.VideoCapture(0)
while(True):
    # Capture the video frame
    ret, frame = video_capture.read()
    # Display the resulting frame
    cv2.imshow('Capturing...', frame)
       # get face region coordinates
    faces , pos = face_detect.get_face(frame)
        # create transparent overlay for bounding box
    # bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    # get face bounding box for overlay
    # for (x,y,w,h)in pos:
    #   bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
    for i in range(len(faces)):
      recog_model = face_detect.recognition_model(faces[i],new_model)
      txt = recog_model[0]
      # print(txt , type(txt))
      font = cv2.FONT_HERSHEY_SIMPLEX
      # fontScale
      fontScale = 1
      # Blue color in BGR
      color = (255, 0, 0)
      # Line thickness of 2 px
      thickness = 2

      x, y, w, h = pos[i]['box']
      # cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
      cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
      cv2.putText(frame,txt,(x,y-10),font,fontScale,color,thickness)
#     # cv2.imwrite('video.mp4',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# # After the loop release the cap object
video_capture.release()
# Destroy all the windows
cv2.destroyAllWindows()

 

detect_time: 0.5966641902923584
all_time: 0.5966649055480957


2024-01-13 09:54:49.716 Python[6526:209499] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


detect_time: 0.5107369422912598
all_time: 0.5107369422912598
detect_time: 0.5011699199676514
all_time: 0.5011699199676514
detect_time: 0.4987647533416748
all_time: 0.4987659454345703
detect_time: 0.6553459167480469
all_time: 0.660728931427002
detect_time: 0.5808460712432861
all_time: 0.5861141681671143
detect_time: 0.5854849815368652
all_time: 0.5910241603851318
detect_time: 0.5854530334472656
all_time: 0.5908870697021484
detect_time: 0.5831160545349121
all_time: 0.5884511470794678
detect_time: 0.580009937286377
all_time: 0.5851180553436279
detect_time: 0.5853660106658936
all_time: 0.590595006942749
detect_time: 0.5692579746246338
all_time: 0.5743420124053955
detect_time: 0.5711789131164551
all_time: 0.576024055480957
detect_time: 0.569951057434082
all_time: 0.5757391452789307
detect_time: 0.5761170387268066
all_time: 0.5816059112548828
detect_time: 0.5908069610595703
all_time: 0.5966160297393799
detect_time: 0.5393261909484863
all_time: 0.5393269062042236
detect_time: 0.53330707550048